In [77]:
pip install lightgbm

Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 2.0 MB 1.6 MB/s eta 0:00:01     |█▉                              | 112 kB 1.4 MB/s eta 0:00:02
You should consider upgrading via the '/home/daiecasquero/anaconda3/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from itertools import cycle, islice
import datetime as datetime
import glob
from matplotlib import pyplot
import plotly.graph_objects as go
import plotly.io as pio
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import log_loss, make_scorer
import category_encoders as ce
from sklearn.ensemble import RandomForestRegressor
import warnings
import lightgbm as lgb

import category_encoders as ce
from sklearn.linear_model import BayesianRidge
from sklearn.cross_decomposition import PLSRegression

%matplotlib inline

plt.style.use('default')
sns.set(style="whitegrid")
pd.options.display.float_format = '{:20,.2f}'.format
warnings.filterwarnings('ignore')
plt.rcParams['figure.figsize'] = (15, 5)

from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
from sklearn import tree
from catboost import CatBoostRegressor

In [3]:
# for preprocessing the data
from sklearn.preprocessing import StandardScaler
# the model
from sklearn.ensemble import RandomForestClassifier
# for combining the preprocess with model training
from sklearn.pipeline import make_pipeline
# for optimizing the hyperparameters of the pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import f1_score

In [4]:
train_labels = pd.read_csv('training_train_labels.csv',low_memory=False, dtype= {
    'damage_grade':'uint8'
}).set_index('building_id')
train_values = pd.read_csv('training_train_values.csv',low_memory=False, dtype= {
    'geo_level_1_id':'category', 
    'geo_level_2_id':'category',
    'geo_level_3_id':'category', 
    'count_floors_pre_eq':'uint8',
    'age':'uint16',
    'area_percentage':'uint16', 
    'height_percentage':'uint16', 
    'land_surface_condition':'category', 
    'foundation_type':'category',
    'roof_type':'category',
    'ground_floor_type':'category',
    'other_floor_type':'category',
    'position':'category',
    'plan_configuration':'category', 
    'has_superstructure_adobe_mud':'uint8',
    'has_superstructure_mud_mortar_stone':'uint8',
    'has_superstructure_stone_flag':'uint8',
    'has_superstructure_cement_mortar_stone':'uint8', 
    'has_superstructure_mud_mortar_brick':'uint8', 
    'has_superstructure_cement_mortar_brick':'uint8', 
    'has_superstructure_timber':'uint8', 
    'has_superstructure_bamboo':'uint8',
    'has_superstructure_rc_non_engineered':'uint8',
    'has_superstructure_rc_engineered':'uint8',
    'has_superstructure_other':'uint8', 
    'legal_ownership_status':'category',
    'count_families':'uint16', 
    'has_secondary_use':'uint8', 
    'has_secondary_use_agriculture':'uint8', 
    'has_secondary_use_hotel':'uint8',
    'has_secondary_use_rental':'uint8',
    'has_secondary_use_institution':'uint8',
    'has_secondary_use_school':'uint8', 
    'has_secondary_use_industry':'uint8', 
    'has_secondary_use_health_post':'uint8', 
    'has_secondary_use_gov_office':'uint8', 
    'has_secondary_use_use_police':'uint8', 
    'has_secondary_use_other':'uint8',
}).set_index('building_id')

pd.options.display.float_format = '{:20,.2f}'.format

In [5]:
train = train_values.merge(train_labels, on='building_id')
train

,geo_level_1_id,geo_level_2_id,geo_level_3_id,count_floors_pre_eq,age,area_percentage,height_percentage,land_surface_condition,foundation_type,roof_type,...,has_secondary_use_hotel,has_secondary_use_rental,has_secondary_use_institution,has_secondary_use_school,has_secondary_use_industry,has_secondary_use_health_post,has_secondary_use_gov_office,has_secondary_use_use_police,has_secondary_use_other,damage_grade
building_id,,,,,,,,,,,,,,,,,,,,,
802906,6,487,12198,2,30,6,5,t,r,n,...,0,0,0,0,0,0,0,0,0,3
28830,8,900,2812,2,10,8,7,o,r,n,...,0,0,0,0,0,0,0,0,0,2
94947,21,363,8973,2,10,5,5,t,r,n,...,0,0,0,0,0,0,0,0,0,3
590882,22,418,10694,2,10,6,5,t,r,n,...,0,0,0,0,0,0,0,0,0,2
201944,11,131,1488,3,30,8,9,t,r,n,...,0,0,0,0,0,0,0,0,0,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1007913,10,1382,11935,2,60,11,6,t,u,x,...,0,0,0,0,0,0,0,0,0,2
349181,13,1154,1044,2,0,8,4,n,r,n,...,0,0,0,0,0,0,0,0,0,1
1009561,26,1001,11008,1,5,15,3,t,i,x,...,0,0,0,0,0,0,0,0,0,2


In [6]:
train = train.drop('geo_level_1_id', axis=1)
train = train.drop('geo_level_2_id', axis=1)
train = train.drop('geo_level_3_id', axis=1)
train = train.drop('land_surface_condition', axis=1)
#train = train.drop('count_floors_pre_eq', axis=1)

In [7]:
#edad promedio segun cantidad de pisos
train['edad_promedio'] = train.groupby('count_floors_pre_eq')['age'].transform('mean')

In [8]:
#cantidad de tipos de techo segun edad
train['tipo_techo'] = train.groupby('age')['roof_type'].transform('count')
#train = train.drop('roof_type', axis=1)

In [9]:
#cantidad de familias por edificacion
train['flias_afectadas'] = train.groupby('building_id')['count_families'].transform('count')

In [10]:
train = train.drop('roof_type', axis=1)
train = train.drop('other_floor_type', axis=1)
train = train.drop('position', axis=1)
#train = train.drop('other_floor_type', axis=1)
train = train.drop('plan_configuration', axis=1)
train = train.drop('legal_ownership_status', axis=1)
train = train.drop('ground_floor_type', axis=1)
train = train.drop('foundation_type', axis=1)

In [11]:
train_value = train['damage_grade']
train_data = train.iloc[:, train.columns != 'damage_grade']

In [12]:
test_labels = pd.read_csv('test_train_labels.csv',low_memory=False, dtype= {
    'damage_grade':'uint8'
}).set_index('building_id')
test_values = pd.read_csv('test_train_values.csv',low_memory=False, dtype= {
    'geo_level_1_id':'category', 
    'geo_level_2_id':'category',
    'geo_level_3_id':'category', 
    'count_floors_pre_eq':'uint8',
    'age':'uint16',
    'area_percentage':'uint16', 
    'height_percentage':'uint16', 
    'land_surface_condition':'category', 
    'foundation_type':'category',
    'roof_type':'category',
    'ground_floor_type':'category',
    'other_floor_type':'category',
    'position':'category',
    'plan_configuration':'category', 
    'has_superstructure_adobe_mud':'uint8',
    'has_superstructure_mud_mortar_stone':'uint8',
    'has_superstructure_stone_flag':'uint8',
    'has_superstructure_cement_mortar_stone':'uint8', 
    'has_superstructure_mud_mortar_brick':'uint8', 
    'has_superstructure_cement_mortar_brick':'uint8', 
    'has_superstructure_timber':'uint8', 
    'has_superstructure_bamboo':'uint8',
    'has_superstructure_rc_non_engineered':'uint8',
    'has_superstructure_rc_engineered':'uint8',
    'has_superstructure_other':'uint8', 
    'legal_ownership_status':'category',
    'count_families':'uint16', 
    'has_secondary_use':'uint8', 
    'has_secondary_use_agriculture':'uint8', 
    'has_secondary_use_hotel':'uint8',
    'has_secondary_use_rental':'uint8',
    'has_secondary_use_institution':'uint8',
    'has_secondary_use_school':'uint8', 
    'has_secondary_use_industry':'uint8', 
    'has_secondary_use_health_post':'uint8', 
    'has_secondary_use_gov_office':'uint8', 
    'has_secondary_use_use_police':'uint8', 
    'has_secondary_use_other':'uint8',
}).set_index('building_id')

pd.options.display.float_format = '{:20,.2f}'.format

In [13]:
test = test_values.merge(test_labels, on='building_id')

In [14]:
test = test.drop('geo_level_1_id', axis=1)
test = test.drop('geo_level_2_id', axis=1)
test = test.drop('geo_level_3_id', axis=1)
test = test.drop('land_surface_condition', axis=1)

In [15]:
#edad promedio segun cantidad de pisos
test['edad_promedio'] = test.groupby('count_floors_pre_eq')['age'].transform('mean')

In [16]:
#cantidad de tipos de techo segun edad
test['tipo_techo'] = test.groupby('age')['roof_type'].transform('count')

In [17]:
#cantidad de familias por edificacion
test['cantidad_flias'] = test.groupby('building_id')['count_families'].transform('count')

In [18]:
test = test.drop('roof_type', axis=1)
test = test.drop('other_floor_type', axis=1)
test = test.drop('position', axis=1)
#test = test.drop('other_floor_type', axis=1)
test = test.drop('plan_configuration', axis=1)
test = test.drop('legal_ownership_status', axis=1)
test = test.drop('ground_floor_type', axis=1)
test = test.drop('foundation_type', axis=1)


test = test.drop_duplicates('age')
#IDs = test[['age']].copy()
IDs = test[['damage_grade']].copy()
test = test.drop('age', axis=1)

In [19]:
test_value = test['damage_grade']
test_data = test.iloc[:, test.columns != 'damage_grade']

In [31]:
modeloBR = BayesianRidge()
LogLoss = make_scorer(log_loss)

In [32]:
parameters = {'n_iter':[10, 50, 100, 300, 500, 1000], 'compute_score':[True, False], 'normalize':[True, False], 'copy_X':[True,False]}
gscv = GridSearchCV(modeloBR, parameters, cv=5, scoring=LogLoss)

In [43]:
train_data

,count_floors_pre_eq,age,area_percentage,height_percentage,has_superstructure_adobe_mud,has_superstructure_mud_mortar_stone,has_superstructure_stone_flag,has_superstructure_cement_mortar_stone,has_superstructure_mud_mortar_brick,has_superstructure_cement_mortar_brick,...,has_secondary_use_institution,has_secondary_use_school,has_secondary_use_industry,has_secondary_use_health_post,has_secondary_use_gov_office,has_secondary_use_use_police,has_secondary_use_other,edad_promedio,tipo_techo,flias_afectadas
building_id,,,,,,,,,,,,,,,,,,,,,
802906,2,30,6,5,1,1,0,0,0,0,...,0,0,0,0,0,0,0,25.82,9299,1
28830,2,10,8,7,0,1,0,0,0,0,...,0,0,0,0,0,0,0,25.82,19889,1
94947,2,10,5,5,0,1,0,0,0,0,...,0,0,0,0,0,0,0,25.82,19889,1
590882,2,10,6,5,0,1,0,0,0,0,...,0,0,0,0,0,0,0,25.82,19889,1
201944,3,30,8,9,1,0,0,0,0,0,...,0,0,0,0,0,0,0,30.18,9299,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1007913,2,60,11,6,0,0,0,0,0,1,...,0,0,0,0,0,0,0,25.82,1809,1
349181,2,0,8,4,0,1,0,0,0,0,...,0,0,0,0,0,0,0,25.82,13307,1
1009561,1,5,15,3,0,0,0,0,0,0,...,0,0,0,0,0,0,0,17.64,17361,1


In [47]:
train_values = train_values.iloc[:133648]
train_values

,geo_level_1_id,geo_level_2_id,geo_level_3_id,count_floors_pre_eq,age,area_percentage,height_percentage,land_surface_condition,foundation_type,roof_type,...,has_secondary_use_agriculture,has_secondary_use_hotel,has_secondary_use_rental,has_secondary_use_institution,has_secondary_use_school,has_secondary_use_industry,has_secondary_use_health_post,has_secondary_use_gov_office,has_secondary_use_use_police,has_secondary_use_other
building_id,,,,,,,,,,,,,,,,,,,,,
802906,6,487,12198,2,30,6,5,t,r,n,...,0,0,0,0,0,0,0,0,0,0
28830,8,900,2812,2,10,8,7,o,r,n,...,0,0,0,0,0,0,0,0,0,0
94947,21,363,8973,2,10,5,5,t,r,n,...,0,0,0,0,0,0,0,0,0,0
590882,22,418,10694,2,10,6,5,t,r,n,...,0,0,0,0,0,0,0,0,0,0
201944,11,131,1488,3,30,8,9,t,r,n,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1007913,10,1382,11935,2,60,11,6,t,u,x,...,0,0,0,0,0,0,0,0,0,0
349181,13,1154,1044,2,0,8,4,n,r,n,...,0,0,0,0,0,0,0,0,0,0
1009561,26,1001,11008,1,5,15,3,t,i,x,...,0,0,0,0,0,0,0,0,0,0


In [50]:
gscv.fit(train_data, train_values)

ValueError: y should be a 1d array, got an array of shape (133648, 38) instead.

In [ ]:
print(gscv.best_score_, gscv.best_estimator_)

In [100]:
modeloBR = BayesianRidge(alpha_1=1e-06, alpha_2=1e-06, compute_score=True, copy_X=True,
              fit_intercept=True, lambda_1=1e-06, lambda_2=1e-06, n_iter=10,
              normalize=False, tol=0.001, verbose=False)
modeloBR.fit(train_data, train_value)
result = modeloBR.predict(test)

entregable_BR = IDs[['damage_grade']].copy()
entregable_BR['Target'] = result

ValueError: Input contains NaN, infinity or a value too large for dtype('float64').

In [ ]:
entregable_BR.to_csv('BR.csv', index=False)

In [52]:
lgb_train = lgb.Dataset(train_data, train_value)
lgb_eval = lgb.Dataset(train_data, train_value, reference=lgb_train)

params = {
    'objective': 'regression',
    'metric': 'logloss',
    'num_leaves': 31,
    'learning_rate': 0.05,
    'feature_fraction': 0.9,
}

params = {
    'objective': 'regression',
}

gbm = lgb.train(params,
                lgb_train,
                num_boost_round=1000,
                valid_sets=lgb_eval,
                early_stopping_rounds=10)

preds = gbm.predict(train_data, num_iteration=gbm.best_iteration)
logLoss = log_loss(train_value.replace(3, 1), preds)
print("logLoss: %f" % (logLoss))

[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.021965 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 227
[LightGBM] [Info] Number of data points in the train set: 133648, number of used features: 27
[LightGBM] [Info] Start training from score 2.237976
[1]	valid_0's l2: 0.36054
Training until validation scores don't improve for 10 rounds
[2]	valid_0's l2: 0.3495
[3]	valid_0's l2: 0.34062
[4]	valid_0's l2: 0.333387
[5]	valid_0's l2: 0.327495
[6]	valid_0's l2: 0.32259
[7]	valid_0's l2: 0.318522
[8]	valid_0's l2: 0.31515
[9]	valid_0's l2: 0.312411
[10]	valid_0's l2: 0.310118
[11]	valid_0's l2: 0.308217
[12]	valid_0's l2: 0.306609
[13]	valid_0's l2: 0.305305
[14]	valid_0's l2: 0.304175
[15]	valid_0's l2: 0.303204
[16]	valid_0's l2: 0.302334
[17]	valid_0's l2: 0.301629
[18]	valid_0's l2: 0.300985
[19]	valid_0's l2: 0.300408
[20]	valid_0'

[284]	valid_0's l2: 0.284531
[285]	valid_0's l2: 0.284513
[286]	valid_0's l2: 0.284492
[287]	valid_0's l2: 0.284477
[288]	valid_0's l2: 0.28446
[289]	valid_0's l2: 0.284417
[290]	valid_0's l2: 0.284385
[291]	valid_0's l2: 0.284356
[292]	valid_0's l2: 0.284297
[293]	valid_0's l2: 0.284258
[294]	valid_0's l2: 0.284229
[295]	valid_0's l2: 0.284206
[296]	valid_0's l2: 0.284169
[297]	valid_0's l2: 0.284157
[298]	valid_0's l2: 0.284119
[299]	valid_0's l2: 0.284081
[300]	valid_0's l2: 0.28404
[301]	valid_0's l2: 0.284003
[302]	valid_0's l2: 0.283983
[303]	valid_0's l2: 0.283955
[304]	valid_0's l2: 0.283937
[305]	valid_0's l2: 0.283906
[306]	valid_0's l2: 0.283879
[307]	valid_0's l2: 0.283826
[308]	valid_0's l2: 0.283796
[309]	valid_0's l2: 0.283761
[310]	valid_0's l2: 0.283747
[311]	valid_0's l2: 0.28373
[312]	valid_0's l2: 0.283689
[313]	valid_0's l2: 0.283654
[314]	valid_0's l2: 0.283631
[315]	valid_0's l2: 0.283614
[316]	valid_0's l2: 0.283595
[317]	valid_0's l2: 0.283565
[318]	valid_0's l

[574]	valid_0's l2: 0.27762
[575]	valid_0's l2: 0.277592
[576]	valid_0's l2: 0.277579
[577]	valid_0's l2: 0.277563
[578]	valid_0's l2: 0.277541
[579]	valid_0's l2: 0.277522
[580]	valid_0's l2: 0.277511
[581]	valid_0's l2: 0.27749
[582]	valid_0's l2: 0.277462
[583]	valid_0's l2: 0.277449
[584]	valid_0's l2: 0.277421
[585]	valid_0's l2: 0.277395
[586]	valid_0's l2: 0.277377
[587]	valid_0's l2: 0.277352
[588]	valid_0's l2: 0.277325
[589]	valid_0's l2: 0.277319
[590]	valid_0's l2: 0.277295
[591]	valid_0's l2: 0.277274
[592]	valid_0's l2: 0.277254
[593]	valid_0's l2: 0.27724
[594]	valid_0's l2: 0.277207
[595]	valid_0's l2: 0.277186
[596]	valid_0's l2: 0.277168
[597]	valid_0's l2: 0.277146
[598]	valid_0's l2: 0.277129
[599]	valid_0's l2: 0.277108
[600]	valid_0's l2: 0.277086
[601]	valid_0's l2: 0.27707
[602]	valid_0's l2: 0.277057
[603]	valid_0's l2: 0.27703
[604]	valid_0's l2: 0.277
[605]	valid_0's l2: 0.276979
[606]	valid_0's l2: 0.276958
[607]	valid_0's l2: 0.276941
[608]	valid_0's l2: 0.

[878]	valid_0's l2: 0.272514
[879]	valid_0's l2: 0.272498
[880]	valid_0's l2: 0.27248
[881]	valid_0's l2: 0.272461
[882]	valid_0's l2: 0.272451
[883]	valid_0's l2: 0.272437
[884]	valid_0's l2: 0.272427
[885]	valid_0's l2: 0.272415
[886]	valid_0's l2: 0.272398
[887]	valid_0's l2: 0.272381
[888]	valid_0's l2: 0.272372
[889]	valid_0's l2: 0.272348
[890]	valid_0's l2: 0.272331
[891]	valid_0's l2: 0.272318
[892]	valid_0's l2: 0.272309
[893]	valid_0's l2: 0.272295
[894]	valid_0's l2: 0.272289
[895]	valid_0's l2: 0.272276
[896]	valid_0's l2: 0.272266
[897]	valid_0's l2: 0.272255
[898]	valid_0's l2: 0.272243
[899]	valid_0's l2: 0.272222
[900]	valid_0's l2: 0.272202
[901]	valid_0's l2: 0.272185
[902]	valid_0's l2: 0.272172
[903]	valid_0's l2: 0.272149
[904]	valid_0's l2: 0.27213
[905]	valid_0's l2: 0.272114
[906]	valid_0's l2: 0.272099
[907]	valid_0's l2: 0.27209
[908]	valid_0's l2: 0.272078
[909]	valid_0's l2: 0.272063
[910]	valid_0's l2: 0.272041
[911]	valid_0's l2: 0.272025
[912]	valid_0's l

In [53]:
result = gbm.predict(test, num_iteration=gbm.best_iteration)

entregable_GBM = IDs[['damage_grade']].copy()
entregable_GBM['Target'] = result